In [1]:
import findspark
findspark.init()
from pyspark import SparkContext

In [2]:
sc = SparkContext("local", "quiz")
import numpy as np 

### 데이터셋
id; age; gender; height; weitht; ap_hi; ap_lo; cholesterol; gluc; smoke; alco; active; cardio

(cardio는 클래스, id는 데이터 인덱스)  
전체 데이터 중 60%는 학습용, 40%는 검증용으로 분리

In [3]:
data = sc.textFile("cardio_train.csv").flatMap(lambda l: l.split('\n')).map(lambda l: l.split(';'))
data = np.array(data.collect()[1:]).astype(np.float)

In [4]:
train_ratio = 0.6
train_size = int(train_ratio*len(data))
valid_size = len(data) - train_size
shuffled_idx = np.random.permutation(len(data))

In [5]:
train = data[shuffled_idx[:train_size]]*0.000001
valid = data[shuffled_idx[train_size:]]*0.000001 # 오버플로우 방지용 범위 조절
D = len(train[0])-1 #feature 

train[:, 0] = 1 # bias(b0)를 위한 feature
valid[:, 0] = 1

train_RDD = sc.parallelize(train)
valid_RDD = sc.parallelize(valid)
print(train[:3])

[[1.0000e+00 1.7628e-02 2.0000e-06 1.7300e-04 1.0000e-04 1.2000e-04
  8.0000e-05 3.0000e-06 1.0000e-06 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00]
 [1.0000e+00 2.0538e-02 1.0000e-06 1.6400e-04 7.1000e-05 1.1000e-04
  7.9000e-05 1.0000e-06 1.0000e-06 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00]
 [1.0000e+00 1.8108e-02 1.0000e-06 1.5500e-04 5.7000e-05 1.1000e-04
  7.0000e-05 1.0000e-06 1.0000e-06 0.0000e+00 0.0000e+00 1.0000e-06
  0.0000e+00]]


In [6]:
import math

maxIter = 50
learning_rate = 0.0001

w = np.random.permutation(D).astype(np.float) # 각 feature의 가중치 벡터

for _ in range(maxIter):
    gradient = train_RDD.map(lambda p: (p[D] - 1/(1+math.exp(-np.dot(w,p[:D]))))*p[:D])\
                    .reduce(lambda a, b: a+b)
    w += learning_rate * np.array(gradient)

print("Final w: {}".format(w))

Final w: [-5.40064715e+00  1.79752972e+00  1.09999860e+01  2.99829054e+00
  6.99922720e+00  9.98658164e-01  9.99898687e+00  3.99998577e+00
  8.99998728e+00  5.99999908e+00  7.99999945e+00 -8.35093315e-06]


### 검증

학습된 가중치와 검증 데이터셋으로 검증

In [7]:
y = valid[:,D]
y_pred = list()

for v in valid:
    y_pred.append(1 / (1 + np.exp(-np.dot(w, v[:D]))))
    
correct_pred = np.equal(np.round(y_pred), y.astype(np.int64))
print("accuracy: ", np.mean(correct_pred.astype(np.float32)))

accuracy:  1.0
